In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Model,Sequential 
from keras.layers import MaxPooling2D
from tensorflow.keras.applications import MobileNetV2

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import PIL

In [7]:
train_dir =r"/kaggle/input/flower-classification-100-classes/train/train"
test_dir = r"/kaggle/input/flower-classification-100-classes/test/test"
val_dir = r"/kaggle/input/flower-classification-100-classes/valid/valid"
Batch_size = 16
Image_size = (224,224)

train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=train_dir,
    batch_size = Batch_size,
    image_size = Image_size,
    seed = 123,
    label_mode = 'int'
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory= val_dir,
    batch_size = Batch_size,
    image_size = Image_size,
    seed = 123,
    label_mode ='int'

)
test_ds = tf.keras.utils.image_dataset_from_directory(
    directory = test_dir,
    batch_size = Batch_size,
    image_size = Image_size,
    seed = 123,
    label_mode ='int'

)

Found 6552 files belonging to 102 classes.


I0000 00:00:1769572481.804284      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1769572481.808190      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 818 files belonging to 102 classes.
Found 819 files belonging to 102 classes.


In [8]:
class_names = train_ds.class_names
print(len(class_names))
print(class_names[:10])


102
['1', '10', '100', '101', '102', '11', '12', '13', '14', '15']


In [ ]:
plt.figure(figsize=(10,10))
for images , labels in train_ds.take(1):
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

    

In [9]:
import json

flower_label_path = r"/kaggle/input/flower-classification-100-classes/Flower-Classes to Flower Names Dictionary.json"


with open(flower_label_path,"r") as f:
    data = json.load(f)

# sort by numeric key
labels_list = [v for k, v in sorted(data.items(), key=lambda x: int(x[0]))]

with open("flower_labels_list.json", "w") as f:
    json.dump(labels_list, f, indent=4)

In [8]:
final_json =r"/kaggle/working/flower_labels_list.json"
with open(final_json,'r') as f:
    final = json.load(f)

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
])


# CNN MOdel

model = tf.keras.Sequential([
    layers.Input(shape=(224,224,3)),
    layers.Rescaling(1./255),   # ← THIS was missing

    data_augmentation,
    layers.Conv2D(32,kernel_size =(3,3), padding="same",activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64,kernel_size =(3,3), padding="same",activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(128,kernel_size =(3,3), padding="same",activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(256,kernel_size =(3,3), padding="same",activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.3),

    layers.GlobalAveragePooling2D(),
    # layers.Flatten(),

    layers.Dense(256,activation= 'relu',
                kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
    layers.Dense(128,activation= 'relu',
                kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
    layers.Dense(102,activation='softmax')
])
model.summary()

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=(1e-3)),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics =['accuracy']
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2,
    restore_best_weights=True
)

model.fit(train_ds,validation_data=val_ds,epochs = 10,callbacks=[early_stopping])

In [ ]:
model.evaluate(test_ds)

In [10]:
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    input_shape=(224,224,3),
    weights ='imagenet'
)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

base_model.trainable = False
inputs = layers.Input(shape=(224,224,3))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x,training = False)

x = layers.BatchNormalization()(x)
x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(128,activation='relu',
                kernel_regularizer = tf.keras.regularizers.l2(1e-3))(x)
x = layers.Dropout(0.2)(x)

outputs = layers.Dense(102,activation='softmax')(x)
model2 = Model(inputs,outputs)
model2.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 7, 7, 1280)     │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 102)            │        13,158 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,440,230 (9.31 MB)

 Trainable params: 179,686 (701.90 KB)

 Non-trainable params: 2,260,544 (8.62 MB)

In [ ]:
model2.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2,
    restore_best_weights=True
)

model2.fit(train_ds,validation_data=val_ds,epochs = 20,callbacks=[early_stopping])

In [ ]:
model2.evaluate(test_ds)

In [ ]:
base_model.trainable=True
for layers in base_model.layers[:50]:
    layers.trainable = False

model2.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate =1e-5),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics =['accuracy']
),
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2,
    restore_best_weights=True
)

model2.fit(train_ds, validation_data= val_ds,epochs=15,callbacks=[early_stopping])


In [ ]:
model2.evaluate(test_ds)

In [ ]:
for images, labels in train_ds.take(1):
    print("labels dtype:", labels.dtype)
    print("labels shape:", labels.shape)
    print("labels sample:", labels[:10])


In [18]:
print(len(train_ds.class_names))


102


In [11]:
base_model = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    input_shape=(224,224,3),
    weights ='imagenet'
)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

base_model.trainable = False
inputs = layers.Input(shape=(224,224,3))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
x = base_model(x,training = False)

x = layers.BatchNormalization()(x)
x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(128,activation='relu',
                kernel_regularizer = tf.keras.regularizers.l2(1e-3))(x)
x = layers.Dropout(0.2)(x)

outputs = layers.Dense(102,activation='softmax')(x)
model_effi = Model(inputs,outputs)
model_effi.summary()

82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-s (Functional)   │ (None, 7, 7, 1280)     │    20,331,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 7, 7, 1280)     │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 102)            │        13,158 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,513,606 (78.25 MB)

 Trainable params: 179,686 (701.90 KB)

 Non-trainable params: 20,333,920 (77.57 MB)

In [13]:
model_effi.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

model_effi.fit(train_ds,validation_data=val_ds,epochs = 20,callbacks=[early_stopping])

Epoch 1/20


E0000 00:00:1769572908.598414      55 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_3_1/efficientnetv2-s_1/block1b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


410/410 ━━━━━━━━━━━━━━━━━━━━ 67s 113ms/step - accuracy: 0.8830 - loss: 0.7340 - val_accuracy: 0.9364 - val_loss: 0.6072
Epoch 2/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 40s 96ms/step - accuracy: 0.8798 - loss: 0.7097 - val_accuracy: 0.9364 - val_loss: 0.5814
Epoch 3/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 39s 96ms/step - accuracy: 0.8825 - loss: 0.7001 - val_accuracy: 0.9279 - val_loss: 0.5957
Epoch 4/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 39s 95ms/step - accuracy: 0.8816 - loss: 0.6979 - val_accuracy: 0.9218 - val_loss: 0.6080
Epoch 5/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 39s 96ms/step - accuracy: 0.8947 - loss: 0.6657 - val_accuracy: 0.9315 - val_loss: 0.5859
Epoch 6/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 40s 97ms/step - accuracy: 0.8973 - loss: 0.6643 - val_accuracy: 0.9328 - val_loss: 0.5762
Epoch 7/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 39s 96ms/step - accuracy: 0.8868 - loss: 0.6827 - val_accuracy: 0.9230 - val_loss: 0.6050
Epoch 8/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 39s 95ms/step - accuracy: 0.8949 - loss: 0.6567 - val_accura

In [14]:
model_effi.evaluate(test_ds)

52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - accuracy: 0.9052 - loss: 0.5838


[0.5736647248268127, 0.9145299196243286]

In [15]:
base_model.trainable=True
for layers in base_model.layers[:50]:
    layers.trainable = False

model_effi.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate =1e-5),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics =['accuracy']
),
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

model_effi.fit(train_ds, validation_data= val_ds,epochs=30,callbacks=[early_stopping])


Epoch 1/30


E0000 00:00:1769573723.229563      55 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_3_1/efficientnetv2-s_1/block1b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


410/410 ━━━━━━━━━━━━━━━━━━━━ 202s 315ms/step - accuracy: 0.4064 - loss: 2.8462 - val_accuracy: 0.7029 - val_loss: 1.3603
Epoch 2/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 123s 300ms/step - accuracy: 0.6152 - loss: 1.7000 - val_accuracy: 0.8105 - val_loss: 0.9984
Epoch 3/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 124s 302ms/step - accuracy: 0.7409 - loss: 1.2077 - val_accuracy: 0.8509 - val_loss: 0.8301
Epoch 4/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 123s 301ms/step - accuracy: 0.7981 - loss: 1.0014 - val_accuracy: 0.8839 - val_loss: 0.7192
Epoch 5/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 123s 301ms/step - accuracy: 0.8388 - loss: 0.8369 - val_accuracy: 0.8936 - val_loss: 0.6649
Epoch 6/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 123s 300ms/step - accuracy: 0.8669 - loss: 0.7498 - val_accuracy: 0.9181 - val_loss: 0.6033
Epoch 7/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 123s 301ms/step - accuracy: 0.8982 - loss: 0.6761 - val_accuracy: 0.9218 - val_loss: 0.5775
Epoch 8/30
410/410 ━━━━━━━━━━━━━━━━━━━━ 123s 300ms/step - accuracy: 0.8950 - loss: 0.65

In [16]:
model_effi.evaluate(test_ds)

52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.9726 - loss: 0.3657


[0.3642879128456116, 0.9731379747390747]

In [17]:
model_effi.save("My_flower_Efficent_model.keras")

In [3]:
import tensorflow as tf

model = tf.keras.models.load_model("My_flower_Efficent_model.keras")


In [4]:
import cv2
import PIL
img_path =r"C:\Users\biswa\OneDrive\Documents\New folder\OneDrive\Desktop\ML Projects\Flower-101 Image Classification Project\yellow lily.jpg"
img = cv2.imread(img_path)
test_img = cv2.resize(img,(224,224))
test_input = test_img.reshape((1,224,224,3))
cv2.imshow("joy",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
import json
with open("class_names.json","r") as f :
    class_type = json.load(f)

In [6]:
class_type

['pink primrose',
 'globe thistle',
 'blanket flower',
 'trumpet creeper',
 'blackberry lily',
 'snapdragon',
 "colt's foot",
 'king protea',
 'spear thistle',
 'yellow iris',
 'globe-flower',
 'purple coneflower',
 'peruvian lily',
 'balloon flower',
 'hard-leaved pocket orchid',
 'giant white arum lily',
 'fire lily',
 'pincushion flower',
 'fritillary',
 'red ginger',
 'grape hyacinth',
 'corn poppy',
 'prince of wales feathers',
 'stemless gentian',
 'artichoke',
 'canterbury bells',
 'sweet william',
 'carnation',
 'garden phlox',
 'love in the mist',
 'mexican aster',
 'alpine sea holly',
 'ruby-lipped cattleya',
 'cape flower',
 'great masterwort',
 'siam tulip',
 'sweet pea',
 'lenten rose',
 'barbeton daisy',
 'daffodil',
 'sword lily',
 'poinsettia',
 'bolero deep blue',
 'wallflower',
 'marigold',
 'buttercup',
 'oxeye daisy',
 'english marigold',
 'common dandelion',
 'petunia',
 'wild pansy',
 'primula',
 'sunflower',
 'pelargonium',
 'bishop of llandaff',
 'gaura',
 'gera

In [24]:
import cv2
import numpy as np

img_path = r"C:\Users\biswa\OneDrive\Documents\New folder\OneDrive\Desktop\ML Projects\Flower-101 Image Classification Project\hibiscus1.jpg"

# Read image
img = cv2.imread(img_path)

# BGR → RGB (IMPORTANT)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Resize
img = cv2.resize(img, (224, 224))

# Add batch dimension ONLY
test_input = np.expand_dims(img, axis=0)

# Predict
prediction = model.predict(test_input)
confidence = float(np.max(prediction))
predicted_index = np.argmax(prediction)
predicted_class = class_type[predicted_index]

print(f"""My prediction is {predicted_class} 
      with confidence {confidence*100:.2f}%""")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
My prediction is hibiscus 
      with confidence 99.91%


In [1]:
%pip uninstall keras -y


Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
print(tf.keras)


<KerasLazyLoader>


In [10]:
print(tf.__version__)


2.15.0


In [1]:
! pip install pipreqs

  Using cached pipreqs-0.5.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached docopt-0.6.2-py2.py3-none-any.whl
  Using cached ipython-8.12.3-py3-none-any.whl.metadata (5.7 kB)
  Using cached yarg-0.1.9-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached backcall-0.2.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached pickleshare-0.7.5-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached pipreqs-0.5.0-py3-none-any.whl (33 kB)
Using cached ipython-8.12.3-py3-none-any.whl (798 kB)
Using cached yarg-0.1.9-py2.py3-none-any.whl (19 kB)
Using cached backcall-0.2.0-py2.py3-none-any.whl (11 kB)
Using cached pickleshare-0.7.5-py2.py3-none-any.whl (6.9 kB)

  Attempting uninstall: ipython

    Found existing installation: ipython 8.38.0

   -------------------------- ------------- 4/6 [ipython]
    Uninstalling ipython-8.38.0:
   -------------------------- ------------- 4/6 [ipython]
      Successfully uninstalled ipython-8.38.0
   -------------------------- ------------- 4/6 [ipython]
   

In [4]:
! pipreqs . --scan-notebooks


Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in .\requirements.txt
